## Построение социальных графов. Меры центральности

**Задание 1 (10 баллов)** 

Прочитайте данные из ```emails.csv```. В колонке ```message``` содержится информация о письме. Вам нужно сформировать несколько таблиц по данным для комфортной работы. 

Во-первых, у Вас имеется информация от том от когдо было отправленно письмо и кому:

* ```From``` - email отправителя
* ```X-From``` - имя отправитела 
* ```To``` - email получателя 
* ```X-To``` - имя получателя

Каждому индивидууму нужно присвоить свой уникальный ```id``` и сформировать таблицу, где по нему можно получить ```email``` и имя.

Также Вам нужно сформировать таблицу, в которой по каждому письму будет содержаться: 

* Дата
* ```id``` отправителя
* ```id``` получателей (обратитие внимание на то, что одно письмо может получать несколько человек)
* Текст

In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [2]:
import pandas as pd
import re
from tqdm import tqdm

In [3]:
emails = pd.read_csv('D:/baklazhan/Compling/programmimng_solutions/python_nlp-master/emails.csv')

In [2]:
#emails = pd.read_csv('C:/Users/ALapidus/Desktop/emails.csv')

In [ ]:
emails.head()

In [5]:
#emails.iloc[412].message.replace('\n\t','').split('\n')
#emails.iloc[86].message.replace('\n\t','').replace('\n ',' ').split('\n')
emails.iloc[86].message

"Message-ID: <12599900.1075855689029.JavaMail.evans@thyme>\nDate: Fri, 25 Aug 2000 03:30:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: brad.mcsherry@enron.com\nSubject: \nCc: john.lavorato@enron.com, hunter.shively@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nBcc: john.lavorato@enron.com, hunter.shively@enron.com\nX-From: Phillip K Allen\nX-To: Brad McSherry\nX-cc: John J Lavorato, Hunter S Shively\nX-bcc: \nX-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf\n\nBrad,\n\n With regard to Tori Kuykendall, I would like to promote her to commercial \nmanager instead of converting her from a commercial support manager to an \nassociate.  Her duties since the beginning of the year have been those of a \ncommercial manager.  I have no doubt that she will compare favorably to \nothers in that category at year end.  \n\n Martin Cuilla on the central desk is in a similiar situat

In [11]:
[emails_sample.iloc[i].message.replace('\n\t','').split('\n')[:18] for i in range(50, 100)]

[['Message-ID: <11798943.1075854281413.JavaMail.evans@thyme>',
  'Date: Thu, 21 Dec 2000 03:43:00 -0800 (PST)',
  'From: mjones7@txu.com',
  'To: daren.j.farmer@enron.com, ggreen2@txu.com, cstone1@txu.com, kenenglish@txu.com, gary.a.hanks@enron.com, earl.tisdale@enron.com, davila1@txu.com, liz.bellamy@enron.com',
  'Subject: HPL Nom for December 22, 2000',
  'Mime-Version: 1.0',
  'Content-Type: text/plain; charset=us-ascii',
  'Content-Transfer-Encoding: 7bit',
  'X-From: "Melissa Jones" <mjones7@txu.com>',
  'X-To: daren.j.farmer@enron.com, "Gary Green" <ggreen2@txu.com>, "Charlie Stone" <cstone1@txu.com>, kenenglish@txu.com, gary.a.hanks@enron.com, earl.tisdale@enron.com, "David Avila" <davila1@txu.com>, Liz.Bellamy@enron.com',
  'X-cc: ',
  'X-bcc: ',
  'X-Folder: \\Darren_Farmer_Jun2001\\Notes Folders\\Discussion threads',
  'X-Origin: Farmer-D',
  'X-FileName: dfarmer.nsf',
  '',
  '(See attached file: HPLN1222.xls)',
  ''],
 ['Message-ID: <29498501.1075845217778.JavaMail.evans@t

In [10]:
emails_sample.iloc[1].message.replace('\n\t','').split('\n')

['Message-ID: <14190314.1075858766117.JavaMail.evans@thyme>',
 'Date: Thu, 18 Oct 2001 16:34:38 -0700 (PDT)',
 'From: fran.chang@enron.com',
 'To: fran.chang@enron.com, heather.dunton@enron.com, samantha.law@enron.com, john.postlethwaite@enron.com, nicholas.warner@enron.com, w..white@enron.com',
 'Subject: ERV Notification:  (West Power Position - 10/18/2001)',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Chang, Fran </O=ENRON/OU=NA/CN=RECIPIENTS/CN=FCHANG>',
 'X-To: Chang, Fran </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Fchang>, Dunton, Heather </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Hdunton>, Law, Samantha </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Slaw>, Postlethwaite, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JPOSTLE>, Warner, Nicholas </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Nwarner>, White, Stacey W. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Swhite>',
 'X-cc: ',
 'X-bcc: ',
 'X-Folder: \\SWHITE (Non-Privileged)\\Deleted Items',
 'X-Origin: White-S',
 'X-File

In [4]:
emails_sample = emails.sample(n = 10000)

In [15]:
def make_contacts_dict(df):
    contacts_dict = {}
    mail_to_list = []
    for index, row in tqdm(df.iterrows()):
        message_split = row['message'].split('\n\n')
        meta = message_split[0]
        text = '\n\n'.join(message_split[1:])
        meta_list = meta.replace('\n\t','').replace('\n ',' ').split('\n')
        for attr in meta_list:
            attr = attr.split(': ')
            if (attr[0] == 'From'):
                mail_from = attr[1]
            if (attr[0] == 'To'):
                mail_to_list = attr[1].split(', ')
            if (attr[0] == 'X-From'):
                name_from = attr[1]
            if (attr[0] == 'X-To'):
                if ('"' in attr[1]):
                    name_to_list = re.findall(r'"(.*?)"', attr[1])
                elif ('</O=ENRON/OU=NA/CN=RECIPIENTS/' in attr[1]):
                    name_to_list = [re.sub(r'.+>,','', name) for name in attr[1].split('</O=ENRON/OU=NA/CN=RECIPIENTS/')]
                else:
                    name_to_list = attr[1].split(', ')

        #print(mail_from, name_from)
        name_from = re.sub(r'<.*?>', '', name_from.replace('@ ENRON', ''))
        contacts_dict.setdefault(mail_from, name_from.strip())

        for mail_to, name_to in zip(mail_to_list, name_to_list):
            name_to = re.sub(r'<.*?>', '', name_to.replace('@ ENRON', ''))
            name_to = re.sub(r'SF40#.+#', '', name_to)
            contacts_dict.setdefault(mail_to, name_to.strip())
            #print(index, mail_to, name_to)
    return contacts_dict

    

In [ ]:
def make_contacts_dict(df):
    contacts_dict = {}
    mail_to_list = []
    for index, row in tqdm(df.iterrows()):
        message_split = row['message'].split('\n\n')
        meta = message_split[0]
        text = '\n\n'.join(message_split[1:])
        meta_list = meta.replace('\n\t','').replace('\n ',' ').split('\n')
        for attr in meta_list:
            attr = attr.split(': ')
            if (attr[0] == 'From'):
                mail_from = attr[1]
            if (attr[0] == 'To'):
                mail_to_list = attr[1].split(', ')
            if (attr[0] == 'X-From'):
                name_from = attr[1]
            if (attr[0] == 'X-To'):
                if ('"' in attr[1]):
                    name_to_list = re.findall(r'"(.*?)"', attr[1])
                elif ('</O=ENRON/OU=NA/CN=RECIPIENTS/' in attr[1]):
                    name_to_list = [re.sub(r'.+>,','', name) for name in attr[1].split('</O=ENRON/OU=NA/CN=RECIPIENTS/')]
                else:
                    name_to_list = attr[1].split(', ')

        #print(mail_from, name_from)
        name_from = re.sub(r'<.*?>', '', name_from.replace('@ ENRON', ''))
        contacts_dict.setdefault(mail_from, name_from.strip())

        for mail_to, name_to in zip(mail_to_list, name_to_list):
            name_to = re.sub(r'<.*?>', '', name_to.replace('@ ENRON', ''))
            name_to = re.sub(r'SF40#.+#', '', name_to)
            contacts_dict.setdefault(mail_to, name_to.strip())
            #print(index, mail_to, name_to)
    return contacts_dict


In [49]:
names = 'Novosel, Sarah </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=E8015336-2D09A27C-862566B9-5830A9>, Alvarez, Ray </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=EBE4476B-2D94882A-86256A14-75FF3B>, Hartsoe, Joe </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=AA8CC789-55F05846-862566B9-583A12>, Bagot, Nancy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=716CAD03-A5CB7D52-86256952-61A958>, Fulton, Donna </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CDE8DAE-A0B9AB37-862566B7-7A1609>, Briggs, Tom </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=179E5C8C-206E9F55-8625691E-7D943>, Nersesian, Carin </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=4CF0E754-2B22EC7C-86256A08-7190CA>, Comnes, Alan'

In [59]:
re.sub('(\w+),? (\w+) </O=ENRON/OU=NA/CN=RECIPIENTS/.+>','\\1\\2', names)
#re.sub('(.+) </O=ENRON/OU=NA/CN=RECIPIENTS/.+>','\\1', names)

'NovoselSarah, Comnes, Alan'

In [50]:
(re.sub('(".+),?(.+")' , '\\1\\2', '"OTC, Dispatch" ghjl')).replace('"','')

'OTC, Dispatch ghjl'

In [13]:
contacts_dict = make_contacts_dict(emails_sample.head(100))

100it [00:00, 2061.74it/s]


In [14]:
contacts_dict

{'a..martin@enron.com': 'Martin, Thomas A.',
 'a..shankman@enron.com': 'Shankman, Jeffrey A.',
 'abb@eslawfirm.com': "'rliebert@cfbf.com'",
 'abh1@pge.com': "'abh1@pge.com'",
 'achan@ladwp.com': "'ronknecht@aol.com'",
 'adam.johnson@enron.com': 'Johnson, Adam',
 'adnan.patel@enron.com': 'Adnan Patel',
 'airam.arteaga@enron.com': 'Airam Arteaga',
 'ajo@cpuc.ca.gov': 'Wetstone, Brad',
 'alan.aronowitz@enron.com': 'Alan Aronowitz',
 'alex.perkins@enron.com': 'Alex Perkins',
 'alex.wong@enron.com': 'Alex Wong',
 'alice.wright@enron.com': 'Alice Wright',
 'alicia.nguyen@enron.com': 'Alicia Nguyen',
 'all.houston@enron.com': 'All Enron Houston',
 'alton.jackson@enron.com': 'Alton Jackson',
 'am1@cpuc.ca.gov': 'Danforth, Christopher',
 'amber.ebow@enron.com': 'Amber Ebow',
 'amy.clemons@enron.com': 'Amy Clemons',
 'amy.horton@enron.com': 'Amy Horton',
 'amy.smith@enron.com': 'Amy Smith',
 'andrea.dahlke@enron.com': 'Andrea Dahlke',
 'andrea.guillen@enron.com': 'Andrea R Guillen',
 'andrea.rin

In [16]:
contacts_dict = make_contacts_dict(emails_sample)

10000it [00:04, 2317.97it/s]


In [17]:
len(contacts_dict)

13177

In [18]:
contacts = pd.DataFrame(data = list(zip(contacts_dict.keys(), contacts_dict.values())), 
                       columns = ['email', 'name'])

In [19]:
contacts = contacts.reset_index()
contacts.columns = ['id', 'email','name']
contacts.head()

,id,email,name
0,0,evan.betzer@enron.com,Evan Betzer
1,1,ronknecht@jmbm.com,ronknecht@JMBM.COM
2,2,stewart.rosman@enron.com,Stewart Rosman
3,3,gregs@mid.org,'gregs@mid.org'
4,4,bgibson50606@yahoo.com,Beverly Gibson


In [10]:
contacts.shape

(13177, 3)

In [31]:
def make_emails_df(raw_emails_df, contacts_df):
    #contacts_dict = {}
    dates = []
    texts = []
    id_from = []
    id_to = []
    #mail_to_list = []
    for index, row in tqdm(raw_emails_df.iterrows()):
        message_split = row['message'].split('\n\n')
        meta = message_split[0]
        text = '\n\n'.join(message_split[1:])
        texts.append(text)
        meta_list = meta.replace('\n\t','').replace('\n ',' ').split('\n')
        for attr in meta_list:
            attr = attr.split(': ')
            if (attr[0] == 'Date'):
                date = attr[1]
            if (attr[0] == 'From'):
                mail_from = attr[1]
            if (attr[0] == 'To'):
                mail_to_list = attr[1].split(', ')
            if (attr[0] == 'X-From'):
                name_from = attr[1]
            if (attr[0] == 'X-To'):
                if ('"' in attr[1]):
                    name_to_list = re.findall(r'"(.*?)"', attr[1])
                else:
                    name_to_list = attr[1].split(', ')

        dates.append(date)
        #id_from.append(int(contacts_df[(contacts_df.email == mail_from)&(contacts_df.name == name_from)].id))
        id_from.append(int(contacts_df[contacts_df.email == mail_from].id))

        #id_to.append([contacts_df[(contacts_df.email == mail_to)&(contacts_df.name == name_to)].id for mail_to, name_to in zip(mail_to_list, name_to_list)])
        id_to_list = []
        for mail_to, name_to in zip(mail_to_list, name_to_list):
            #print(contacts_df[(contacts_df.email == mail_to)&(contacts_df.name == name_to)].id)
            #print(mail_to, name_to)
            #id_to_list.append(str(int(contacts_df[(contacts_df.email == mail_to)&(contacts_df.name == name_to)].id)))
            print(mail_to, contacts_df[contacts_df.email == mail_to].id)
            id_to_list.append(str(int(contacts_df[contacts_df.email == mail_to].id)))        
        id_to.append(','.join(id_to_list))
        
            #contacts_dict.setdefault(mail_to, name_to)
            #print(index, mail_to, name_to)
    emails_df = pd.DataFrame({'date':dates, 'id_from':id_from, 'id_to':id_to, 'text':texts})
    return emails_df

In [46]:
for index, row in emails_sample.iterrows():
    message_split = row['message'].split('\n\n')
    meta = message_split[0]
    #text = '\n\n'.join(message_split[1:])
    #texts.append(text)
    meta_list = meta.replace('\n\t','').replace('\n ',' ').split('\n')
    for attr in meta_list:
        attr = attr.split(': ')
        if (attr[0] == 'To'):
            if ('sscott3@enron.com, davef@jmbm.com' in attr[1]):
                print(index, meta)

407292 Message-ID: <18917591.1075846693478.JavaMail.evans@thyme>
Date: Mon, 31 Jul 2000 08:24:00 -0700 (PDT)
From: al2@jmbm.com
To: sscott3@enron.com, davef@jmbm.com, garyb@jmbm.com, mdjoseph@jmbm.com, 
	epoole@jmbm.com, chj@jmbm.com, eke@jmbm.com, haorndorff@jmbm.com, 
	kbhensman@jmbm.com, jweil@jmbm.com, pjpowerlaw@jmbm.com, 
	rczahar@jmbm.com, ronknecht@jmbm.com, johnj@jmbm.com, 
	craigc@jmbm.com, kolling@jmbm.com, maan@jmbm.com, kmills@jmbm.com, 
	bjeider@jmbm.com, slins@jmbm.com, eklinkner@jmbm.com, ed@jmbm.com, 
	rfoss@jmbm.com, alb@jmbm.com, bho@jmbm.com, edf@jmbm.com, 
	eyq@jmbm.com, jwr@jmbm.com, ltt@jmbm.com, mcn@jmbm.com, plg@jmbm.com, 
	ram@jmbm.com, raw@jmbm.com, rmp@jmbm.com, sbs@jmbm.com, tah@jmbm.com, 
	vjb@jmbm.com, tomb@jmbm.com, burkee@jmbm.com, rochmanm@jmbm.com, 
	dcarroll@jmbm.com, lindseyhowdowning@jmbm.com, salleyoo@jmbm.com, 
	gtbl@jmbm.com, jmpa@jmbm.com, ceyap@jmbm.com, furutanj@jmbm.com, 
	askaff@jmbm.com, bwood@jmbm.com, jtachera@jmbm.com, mjaske@jmbm.com, 

In [34]:
contacts_dict['constantine.brian@enron.com']

KeyError: 'constantine.brian@enron.com'

In [32]:
emails_df = make_emails_df(emails_sample, contacts)


0it [00:00, ?it/s]

m..love@enron.com 698    698
Name: id, dtype: int64
scott.palmer@enron.com 10445    10445
Name: id, dtype: int64
c..giron@enron.com 1505    1505
Name: id, dtype: int64
d..winfree@enron.com 3329    3329
Name: id, dtype: int64


1it [00:00,  3.51it/s]

xdymyljedukkbynn@msn.com 7115    7115
Name: id, dtype: int64
ying.liu@enron.com 4062    4062
Name: id, dtype: int64


3it [00:00,  6.55it/s]

...

george.mcclellan@enron.com 2833    2833
Name: id, dtype: int64
michael.beyer@enron.com 3204    3204
Name: id, dtype: int64
daniel.reck@enron.com 9090    9090
Name: id, dtype: int64
kevin.mcgowan@enron.com 11430    11430
Name: id, dtype: int64
christopher.helfrich@enron.com 10104    10104
Name: id, dtype: int64
robert.jones@enron.com 8046    8046
Name: id, dtype: int64


198it [00:37,  5.25it/s]

fenner.chet@enron.com 7957    7957
Name: id, dtype: int64
corrier.brad@enron.com 6416    6416
Name: id, dtype: int64
joe.parks@enron.com 641    641
Name: id, dtype: int64
wollam.erik@enron.com 5667    5667
Name: id, dtype: int64
constantine.brian@enron.com Series([], Name: id, dtype: int64)


TypeError: cannot convert the series to <class 'int'>

198it [00:50,  3.95it/s]

In [10]:
emails_df.head(20)

,date,id_from,id_to,text
0,"Mon, 22 Oct 2001 12:18:29 -0700 (PDT)",0,"1,2","\nFranky,\n\nPlease delegate this project and ..."
1,"Tue, 8 May 2001 13:50:00 -0700 (PDT)",3,4,\n---------------------- Forwarded by Chris Ge...
2,"Fri, 20 Apr 2001 08:05:00 -0700 (PDT)",5,6,I think Mike should sign\n\n\n\n\nPeter Styles...
3,"Fri, 21 Sep 2001 13:08:44 -0700 (PDT)",7,8,Dear Mark:\n\nTest successful!!!!\n\nIf you wa...
4,"Mon, 27 Mar 2000 08:14:00 -0800 (PST)",9,"10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25",Enron Methanol nominates the following natural...
5,"Fri, 13 Apr 2001 14:43:00 -0700 (PDT)",26,27,"David,\n\nThanks for the invitation. \nI shall..."
6,"Mon, 26 Nov 2001 06:43:13 -0800 (PST)",28,29,The word is - the end is near. Look for somet...
7,"Fri, 14 Apr 2000 11:56:00 -0700 (PDT)",30,5,Steven\n\nI will fax you now the revised Execu...
8,"Mon, 19 Nov 2001 05:12:09 -0800 (PST)",31,5,\n\t\nEnron in Action can be accessed through ...
9,"Tue, 12 Dec 2000 17:57:00 -0800 (PST)",32,"33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48","Hey Y'all,\n\nLess then 2 weekes before I'm ba..."


In [96]:
emails_df[emails_df.id_to == '']

,date,id_from,id_to,text
0,"Mon, 9 Apr 2001 06:14:51 -0700 (PDT)",0,,Set up by Julie Armstrong for McCarty/33597
2838,"Wed, 12 Dec 2001 07:36:59 -0800 (PST)",2920,,"\n\n -----Original Message-----\nFrom: \tLove,..."
7385,"Tue, 23 Apr 2002 10:01:00 -0700 (PDT)",11619,,"Clemens,\n\nI agree. for our immediate needs, ..."


In [11]:
emails_df[emails_df.id_to == '']

,date,id_from,id_to,text


In [97]:
emails_df = emails_df[emails_df.id_to != '']

In [98]:
emails_df = emails_df.reset_index(drop=True)

In [12]:
emails_df.shape

(10000, 4)

In [13]:
contacts.shape

(12883, 3)

**Задание 2 (15 баллов)** 

Сформируйте матрицу $A = (a_{ij})_{i,j = 1}^N$, где $a_{ij}$ - количество писем, которое индивид $i$ отправил индивиду $j$, $N$ - количество индивидуумов

Далее из матрицы $A$ получим симметричную матрицу, которая будет показывать нам частоту коммуникаций между каждой парой индивидов: $A+A^T$. Постройте по данной матрице не направленный граф, где узлы должны называться по именам, которые вы извлекли из ```X-From``` и ```X-To```

**Замечание.** Так как граф слишком большой, то вы можете оставить только наиболее активных участников.

In [14]:
import numpy as np

In [15]:
[int(mail) for mail in emails_df.iloc[1].id_to.split(',')]

[4]

In [16]:
def make_communication_matrix(df, contacts):
    N = contacts.shape[0]
    A = np.zeros((N,N))
    
    for index, row in df.iterrows():
        from_i = row['id_from']
        for to_j in row['id_to'].split(','):
            A[from_i][int(to_j)] += 1
    return A + np.transpose(A)
        

In [17]:
%%time
communication_matrix = make_communication_matrix(emails_df, contacts)

Wall time: 28.1 s


In [18]:
communication_matrix.shape

(12883, 12883)

In [19]:
communication_matrix.max()

386.0

In [20]:
communication_matrix.mean()

0.00071712230391292819

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
#plt.matshow(communication_matrix)
#plt.colorbar()

#plt.xticks(np.arange(N), le.classes_, rotation=90)
#plt.yticks(np.arange(N), le.classes_)

In [22]:
import networkx as nx

**Задание 3 (25 баллов)**

Оцените важность отдельных узлов, используя следующие меры:

* Degree centrality
* Closeness centrality
* Betweenness centrality
* Eigenvector centrality

Подробнее читайте в статье: УЗЛЫ В СОЦИАЛЬНЫХ СЕТЯХ: МЕРЫ ЦЕНТРАЛЬНОСТИ И РОЛЬ В СЕТЕВЫХ ПРОЦЕССАХ

Для каждой меры центральности выведите топ пользователей

**Бонусное задание (25 баллов)** 

Выделите темы, которые обсуждаются в письмах.

Выберите одну из мер центральности и для топа пользователей по данной мере укажите, какие темы они обсуждают чаще всего и с кем.